In [434]:
import pandas as pd
pd.set_option('display.max_columns', None)

# Verison 1.0

In [435]:
df = pd.read_csv("/home/redha/Documents/projects/NLP/sociolla/data/archive/products_all_brands.csv")

df.describe()

,product_id,beauty_point_earned,average_rating,total_reviews,total_recommended_count,total_repurchase_maybe_count,total_repurchase_no_count,total_repurchase_yes_count,total_in_wishlist
count,7636.000000,7636.000000,7636.000000,7636.000000,7636.000000,7636.000000,7636.000000,7636.000000,7636.000000
mean,82903.974725,37.236773,3.517905,198.197617,186.560503,40.603457,15.479963,141.873494,633.016108
std,27257.173650,46.056962,2.000138,852.067365,803.185318,169.473278,79.716058,625.673542,2055.067002
min,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-317.000000
25%,75267.000000,10.000000,3.910714,1.000000,1.000000,0.000000,0.000000,0.000000,28.000000
50%,91856.500000,20.000000,4.576923,9.000000,9.000000,2.000000,0.000000,6.000000,132.000000
75%,101825.500000,50.000000,4.738155,76.000000,71.000000,17.000000,6.000000,52.000000,480.000000
max,109155.000000,610.000000,5.000000,21536.000000,20804.000000,5025.000000,2824.000000,17653.000000,58568.000000


In [436]:
df.shape

(7636, 19)

In [437]:
df.drop(columns=['price_by_combinations','url'], inplace=True)

In [438]:
df.duplicated().sum()

0

In [439]:
df.drop_duplicates(inplace=True)


In [440]:
df.isna().sum()

brand_name                         0
product_name                       0
product_id                         0
beauty_point_earned                0
price_range                        0
active_date                     2102
default_category                   0
categories                         4
rating_types_str                  75
average_rating                     0
total_reviews                      0
average_rating_by_types         1846
total_recommended_count            0
total_repurchase_maybe_count       0
total_repurchase_no_count          0
total_repurchase_yes_count         0
total_in_wishlist                  0
dtype: int64

In [441]:
df.dropna(subset=['active_date'], inplace=True)
df.dropna(subset=['categories'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [442]:
df

,brand_name,product_name,product_id,beauty_point_earned,price_range,active_date,default_category,categories,rating_types_str,average_rating,total_reviews,average_rating_by_types,total_recommended_count,total_repurchase_maybe_count,total_repurchase_no_count,total_repurchase_yes_count,total_in_wishlist
0,796_3ce,MULTI EYE COLOR PALETTE,97802,130,Rp 555.000 - Rp 687.000,2022-10-03T03:30:06.681Z,Eyeshadow,Makeup; Eyes; Eyeshadow,is_star_long_wear;is_star_packaging;is_star_pi...,4.920000,5,"""star_long_wear"": 5; ""star_packaging"": 4.8; ""s...",5,0,0,5,717
1,796_3ce,VELVET LIP TINT,97810,50,Rp 264.000,2022-10-03T03:02:40.340Z,Lip Cream,Makeup; Lips; Lip Cream,is_star_long_wear;is_star_packaging;is_star_pi...,4.576190,42,"""star_long_wear"": 4.309523809523809; ""star_pac...",42,10,2,30,682
2,796_3ce,LIP COLOR,97822,60,Rp 317.000,2023-05-30T09:49:15.158Z,Lipstick,Makeup; Lips; Lipstick,is_star_long_wear;is_star_packaging;is_star_pi...,0.000000,0,NaN,0,0,0,0,173
3,796_3ce,MINI MULTI EYE COLOR PALETTE,97833,80,Rp 423.000,2022-10-03T03:27:01.334Z,Eyeshadow,Makeup; Eyes; Eyeshadow,is_star_long_wear;is_star_packaging;is_star_pi...,4.883333,6,"""star_long_wear"": 4.916666666666667; ""star_pac...",12,1,0,11,257
4,796_3ce,FACE BLUSH,97801,60,Rp 300.000,2022-10-03T03:22:26.610Z,Blush,Makeup; Face; Blush,is_star_long_wear;is_star_packaging;is_star_pi...,4.858824,13,"""star_long_wear"": 4.9411764705882355; ""star_pa...",17,2,0,15,387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5528,15371_youvit,Youvit Acno 7 days,103726,0,Rp 29.900,2023-06-06T02:47:32.800Z,Beauty Suplements,Supplement; Beauty Suplements,is_star_effectiveness;is_star_eficiency;is_sta...,4.333333,6,"""star_effectiveness"": 4.166666666666667; ""star...",4,3,0,3,25
5529,273_yves-rocher,Shower Gel,63865,30,Rp 119.000 - Rp 179.000,2023-09-28T09:37:13.539Z,Body Wash,Bath & Body; Body Cleanser; Body Wash,is_star_effectiveness;is_star_packaging;is_sta...,4.660714,14,"""star_effectiveness"": 4.857142857142857; ""star...",14,4,0,10,332
5530,273_yves-rocher,Relaxing Shower Gel Olive Petitgrain,68293,20,Rp 119.000,2023-04-13T07:35:43.813Z,Body Wash,Chăm sóc cơ thể; Làm sạch; Sữa tắm,is_star_effectiveness;is_star_packaging;is_sta...,4.607143,7,"""star_effectiveness"": 4.571428571428571; ""star...",7,1,1,5,171
5531,273_yves-rocher,Repair Hair Mask,36357,50,Rp 289.000,2022-10-27T07:33:19.380Z,Hair Mask,Hair Care; Hair Treatment; Hair Mask,is_star_effectiveness;is_star_packaging;is_sta...,4.750000,5,"""star_effectiveness"": 5; ""star_packaging"": 5; ...",5,1,0,4,58


In [443]:
# process prince range
df["price_range"] = (
    df["price_range"]
    .str.replace('Rp', '')
    .str.replace('.', '')
    .str.replace(' ', '')
    .apply(lambda x: sum(map(int, x.split('-'))) / len(x.split('-')))
)

In [444]:
df

,brand_name,product_name,product_id,beauty_point_earned,price_range,active_date,default_category,categories,rating_types_str,average_rating,total_reviews,average_rating_by_types,total_recommended_count,total_repurchase_maybe_count,total_repurchase_no_count,total_repurchase_yes_count,total_in_wishlist
0,796_3ce,MULTI EYE COLOR PALETTE,97802,130,621000.0,2022-10-03T03:30:06.681Z,Eyeshadow,Makeup; Eyes; Eyeshadow,is_star_long_wear;is_star_packaging;is_star_pi...,4.920000,5,"""star_long_wear"": 5; ""star_packaging"": 4.8; ""s...",5,0,0,5,717
1,796_3ce,VELVET LIP TINT,97810,50,264000.0,2022-10-03T03:02:40.340Z,Lip Cream,Makeup; Lips; Lip Cream,is_star_long_wear;is_star_packaging;is_star_pi...,4.576190,42,"""star_long_wear"": 4.309523809523809; ""star_pac...",42,10,2,30,682
2,796_3ce,LIP COLOR,97822,60,317000.0,2023-05-30T09:49:15.158Z,Lipstick,Makeup; Lips; Lipstick,is_star_long_wear;is_star_packaging;is_star_pi...,0.000000,0,NaN,0,0,0,0,173
3,796_3ce,MINI MULTI EYE COLOR PALETTE,97833,80,423000.0,2022-10-03T03:27:01.334Z,Eyeshadow,Makeup; Eyes; Eyeshadow,is_star_long_wear;is_star_packaging;is_star_pi...,4.883333,6,"""star_long_wear"": 4.916666666666667; ""star_pac...",12,1,0,11,257
4,796_3ce,FACE BLUSH,97801,60,300000.0,2022-10-03T03:22:26.610Z,Blush,Makeup; Face; Blush,is_star_long_wear;is_star_packaging;is_star_pi...,4.858824,13,"""star_long_wear"": 4.9411764705882355; ""star_pa...",17,2,0,15,387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5528,15371_youvit,Youvit Acno 7 days,103726,0,29900.0,2023-06-06T02:47:32.800Z,Beauty Suplements,Supplement; Beauty Suplements,is_star_effectiveness;is_star_eficiency;is_sta...,4.333333,6,"""star_effectiveness"": 4.166666666666667; ""star...",4,3,0,3,25
5529,273_yves-rocher,Shower Gel,63865,30,149000.0,2023-09-28T09:37:13.539Z,Body Wash,Bath & Body; Body Cleanser; Body Wash,is_star_effectiveness;is_star_packaging;is_sta...,4.660714,14,"""star_effectiveness"": 4.857142857142857; ""star...",14,4,0,10,332
5530,273_yves-rocher,Relaxing Shower Gel Olive Petitgrain,68293,20,119000.0,2023-04-13T07:35:43.813Z,Body Wash,Chăm sóc cơ thể; Làm sạch; Sữa tắm,is_star_effectiveness;is_star_packaging;is_sta...,4.607143,7,"""star_effectiveness"": 4.571428571428571; ""star...",7,1,1,5,171
5531,273_yves-rocher,Repair Hair Mask,36357,50,289000.0,2022-10-27T07:33:19.380Z,Hair Mask,Hair Care; Hair Treatment; Hair Mask,is_star_effectiveness;is_star_packaging;is_sta...,4.750000,5,"""star_effectiveness"": 5; ""star_packaging"": 5; ...",5,1,0,4,58


In [445]:
df[['categories1', 'categories2', 'categories3']] = df['categories'].str.split(';', expand=True).apply(lambda x: x.str.strip())

In [446]:
df['categories1'].unique()

array(['Makeup', 'Skincare', 'Accessories', 'Fragrance', 'Bath & Body',
       'Hair Care', 'Body Oil', 'Chăm sóc da', 'Nail Care', 'Supplement',
       'Treatment', 'Sanitary', 'Oral Care', 'Chăm sóc cơ thể',
       'Home Care', 'Gift Set', 'Shaving & Grooming',
       'Bedding & Bathroom', 'Cleanser', 'Chăm sóc tóc', 'Moisturizer',
       'Nursing & Feeding', 'Make Up - Not Used', 'Miscellaneous'],
      dtype=object)

In [447]:
df['categories2'].unique()

array(['Eyes', 'Lips', 'Face', 'Sun Care', 'Skin Care Set', 'Treatment',
       'Face Mask', 'Moisturizer', 'Cleanser', 'Lip Care', None,
       'Eye Care', 'Makeup Brushes', 'Others', 'Facial', 'Makeup Tools',
       'Makeup Palettes & Set', 'Organizers', 'Body Fragrance',
       'Body Care', 'Sunscreen', 'Shampoo', 'Hair Care Set',
       'Hair Treatment', 'Bath & Body', 'Hand & Foot Cream',
       'Body Cleanser', 'Set Chăm sóc da', 'Nail Beauty', 'Supplement',
       'Bath & Body Set', 'Làm sạch', 'Body Scrub & Exfoliants',
       'Essence', 'Makeup Remover', 'Body Treatment', 'Make up set',
       'Feminine sanitary', 'Oral Care', 'Dưỡng thể', 'Hair Styling',
       'Chăm sóc chuyên sâu', 'Chăm sóc mắt', 'Home Care',
       'Gift Set For The Family', 'Home Fragrance', 'Bug Repellent',
       'Manicure & Pedicure', 'Hair Tools', 'Mask', 'Shaving & Grooming',
       'Shaver and Clipper', 'Shaving Cream & Gel', 'Bath', 'Face Wash',
       'Cleansing Gel', 'Toner', 'Mặt nạ', 'Baby Wip

In [448]:
df['categories3'].unique()

array(['Eyeshadow', 'Lip Cream', 'Lipstick', 'Blush', 'Lip Tint',
       'Eyebrows', 'Foundation', 'Mascara', 'Pressed Powder', 'Eyeliner',
       'Sunscreen', None, 'Toner', 'Essence', 'Sheet Mask',
       'Face Cream & Lotion', 'Face Wash', 'Wash off Mask', 'Lip Balm',
       'Cleansing Bar', 'Cleansing Gel', 'Face Serum', 'Skin Care Set',
       'Eye Cream', 'Ampoule', 'Acne Treatment / Sealing gel / Cream',
       'Concealer', 'Sponge & Applicators', 'Makeup Pouch',
       'Face Brushes', 'Facial Tools', 'Brow Tools',
       'Travel Bottles & Makeup case', 'Brush Cleanser & Tools',
       'Brush Set', 'Eau De Parfum', 'Body Lotion / Body Serum',
       'Cushion', 'Sun Care', 'Tinted Moisturizer', 'Shampoo',
       'Hair Care Set', 'Hair Oil', 'Body Care', 'Clay Mask',
       'Cleansing Wipes', 'Micellar Water', 'Face Mist', 'Sleeping Mask',
       'Hand & Foot Cream', 'False Eyelashes', 'Body Wash', 'Eye Serum',
       'Anti-Aging', 'Face Gel', 'Set Chăm sóc da', 'Makeup Remover',


In [449]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5533 entries, 0 to 5532
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   brand_name                    5533 non-null   object 
 1   product_name                  5533 non-null   object 
 2   product_id                    5533 non-null   int64  
 3   beauty_point_earned           5533 non-null   int64  
 4   price_range                   5533 non-null   float64
 5   active_date                   5533 non-null   object 
 6   default_category              5533 non-null   object 
 7   categories                    5533 non-null   object 
 8   rating_types_str              5480 non-null   object 
 9   average_rating                5533 non-null   float64
 10  total_reviews                 5533 non-null   int64  
 11  average_rating_by_types       4085 non-null   object 
 12  total_recommended_count       5533 non-null   int64  
 13  tot

In [450]:
for index, row in df.iterrows():
    if pd.isnull(row['categories2']):
        df.at[index, 'categories2'] = row['categories1']

    if pd.isnull(row['categories3']):
        df.at[index, 'categories3'] = row['categories1']

df.reset_index(drop=True, inplace=True)

In [451]:
df.drop('categories', axis=1, inplace=True)

In [452]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5533 entries, 0 to 5532
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   brand_name                    5533 non-null   object 
 1   product_name                  5533 non-null   object 
 2   product_id                    5533 non-null   int64  
 3   beauty_point_earned           5533 non-null   int64  
 4   price_range                   5533 non-null   float64
 5   active_date                   5533 non-null   object 
 6   default_category              5533 non-null   object 
 7   rating_types_str              5480 non-null   object 
 8   average_rating                5533 non-null   float64
 9   total_reviews                 5533 non-null   int64  
 10  average_rating_by_types       4085 non-null   object 
 11  total_recommended_count       5533 non-null   int64  
 12  total_repurchase_maybe_count  5533 non-null   int64  
 13  tot

In [453]:
df

,brand_name,product_name,product_id,beauty_point_earned,price_range,active_date,default_category,rating_types_str,average_rating,total_reviews,average_rating_by_types,total_recommended_count,total_repurchase_maybe_count,total_repurchase_no_count,total_repurchase_yes_count,total_in_wishlist,categories1,categories2,categories3
0,796_3ce,MULTI EYE COLOR PALETTE,97802,130,621000.0,2022-10-03T03:30:06.681Z,Eyeshadow,is_star_long_wear;is_star_packaging;is_star_pi...,4.920000,5,"""star_long_wear"": 5; ""star_packaging"": 4.8; ""s...",5,0,0,5,717,Makeup,Eyes,Eyeshadow
1,796_3ce,VELVET LIP TINT,97810,50,264000.0,2022-10-03T03:02:40.340Z,Lip Cream,is_star_long_wear;is_star_packaging;is_star_pi...,4.576190,42,"""star_long_wear"": 4.309523809523809; ""star_pac...",42,10,2,30,682,Makeup,Lips,Lip Cream
2,796_3ce,LIP COLOR,97822,60,317000.0,2023-05-30T09:49:15.158Z,Lipstick,is_star_long_wear;is_star_packaging;is_star_pi...,0.000000,0,NaN,0,0,0,0,173,Makeup,Lips,Lipstick
3,796_3ce,MINI MULTI EYE COLOR PALETTE,97833,80,423000.0,2022-10-03T03:27:01.334Z,Eyeshadow,is_star_long_wear;is_star_packaging;is_star_pi...,4.883333,6,"""star_long_wear"": 4.916666666666667; ""star_pac...",12,1,0,11,257,Makeup,Eyes,Eyeshadow
4,796_3ce,FACE BLUSH,97801,60,300000.0,2022-10-03T03:22:26.610Z,Blush,is_star_long_wear;is_star_packaging;is_star_pi...,4.858824,13,"""star_long_wear"": 4.9411764705882355; ""star_pa...",17,2,0,15,387,Makeup,Face,Blush
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5528,15371_youvit,Youvit Acno 7 days,103726,0,29900.0,2023-06-06T02:47:32.800Z,Beauty Suplements,is_star_effectiveness;is_star_eficiency;is_sta...,4.333333,6,"""star_effectiveness"": 4.166666666666667; ""star...",4,3,0,3,25,Supplement,Beauty Suplements,Supplement
5529,273_yves-rocher,Shower Gel,63865,30,149000.0,2023-09-28T09:37:13.539Z,Body Wash,is_star_effectiveness;is_star_packaging;is_sta...,4.660714,14,"""star_effectiveness"": 4.857142857142857; ""star...",14,4,0,10,332,Bath & Body,Body Cleanser,Body Wash
5530,273_yves-rocher,Relaxing Shower Gel Olive Petitgrain,68293,20,119000.0,2023-04-13T07:35:43.813Z,Body Wash,is_star_effectiveness;is_star_packaging;is_sta...,4.607143,7,"""star_effectiveness"": 4.571428571428571; ""star...",7,1,1,5,171,Chăm sóc cơ thể,Làm sạch,Sữa tắm
5531,273_yves-rocher,Repair Hair Mask,36357,50,289000.0,2022-10-27T07:33:19.380Z,Hair Mask,is_star_effectiveness;is_star_packaging;is_sta...,4.750000,5,"""star_effectiveness"": 5; ""star_packaging"": 5; ...",5,1,0,4,58,Hair Care,Hair Treatment,Hair Mask


In [454]:
def extract_ratings(value):
    try:
        elements = value.split(';')
        ratings = {}
        for element in elements:
            sub_elements = element.split(':')
            key = sub_elements[0].strip(' "')
            value = float(sub_elements[1])
            ratings[key] = value
        
        return ratings
    except:
        return {
            'star_value_for_money': 0,
            'star_long_wear': 0,
            'star_texture': 0,
            'star_quality': 0,
            'star_eficiency': 0,
            'star_durability': 0,
            'star_scent': 0,
            'star_pigmentation': 0,
            'star_packaging': 0,
            'star_effectiveness': 0
        }

default_values = {
    'star_value_for_money': 0,
    'star_long_wear': 0,
    'star_texture': 0,
    'star_quality': 0,
    'star_eficiency': 0,
    'star_durability': 0,
    'star_scent': 0,
    'star_pigmentation': 0,
    'star_packaging': 0,
    'star_effectiveness': 0
}

df['ratings'] = df['average_rating_by_types'].apply(lambda x: {**default_values, **extract_ratings(x)})

df = pd.concat([df, df['ratings'].apply(pd.Series)], axis=1)



In [455]:
df = df.drop('ratings', axis=1)
df.drop(columns=['average_rating_by_types','rating_types_str'],axis=1, inplace=True)

In [456]:
df

,brand_name,product_name,product_id,beauty_point_earned,price_range,active_date,default_category,average_rating,total_reviews,total_recommended_count,total_repurchase_maybe_count,total_repurchase_no_count,total_repurchase_yes_count,total_in_wishlist,categories1,categories2,categories3,star_value_for_money,star_long_wear,star_texture,star_quality,star_eficiency,star_durability,star_scent,star_pigmentation,star_packaging,star_effectiveness
0,796_3ce,MULTI EYE COLOR PALETTE,97802,130,621000.0,2022-10-03T03:30:06.681Z,Eyeshadow,4.920000,5,5,0,0,5,717,Makeup,Eyes,Eyeshadow,5.000000,5.000000,5.000000,0.0,0.0,0.0,0.0,4.800000,4.800000,0.000000
1,796_3ce,VELVET LIP TINT,97810,50,264000.0,2022-10-03T03:02:40.340Z,Lip Cream,4.576190,42,42,10,2,30,682,Makeup,Lips,Lip Cream,4.333333,4.309524,4.761905,0.0,0.0,0.0,0.0,4.690476,4.785714,0.000000
2,796_3ce,LIP COLOR,97822,60,317000.0,2023-05-30T09:49:15.158Z,Lipstick,0.000000,0,0,0,0,0,173,Makeup,Lips,Lipstick,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
3,796_3ce,MINI MULTI EYE COLOR PALETTE,97833,80,423000.0,2022-10-03T03:27:01.334Z,Eyeshadow,4.883333,6,12,1,0,11,257,Makeup,Eyes,Eyeshadow,4.750000,4.916667,4.916667,0.0,0.0,0.0,0.0,4.833333,5.000000,0.000000
4,796_3ce,FACE BLUSH,97801,60,300000.0,2022-10-03T03:22:26.610Z,Blush,4.858824,13,17,2,0,15,387,Makeup,Face,Blush,4.764706,4.941176,4.882353,0.0,0.0,0.0,0.0,4.705882,5.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5528,15371_youvit,Youvit Acno 7 days,103726,0,29900.0,2023-06-06T02:47:32.800Z,Beauty Suplements,4.333333,6,4,3,0,3,25,Supplement,Beauty Suplements,Supplement,4.333333,0.000000,0.000000,0.0,4.5,0.0,0.0,0.000000,0.000000,4.166667
5529,273_yves-rocher,Shower Gel,63865,30,149000.0,2023-09-28T09:37:13.539Z,Body Wash,4.660714,14,14,4,0,10,332,Bath & Body,Body Cleanser,Body Wash,4.571429,0.000000,4.785714,0.0,0.0,0.0,0.0,0.000000,4.428571,4.857143
5530,273_yves-rocher,Relaxing Shower Gel Olive Petitgrain,68293,20,119000.0,2023-04-13T07:35:43.813Z,Body Wash,4.607143,7,7,1,1,5,171,Chăm sóc cơ thể,Làm sạch,Sữa tắm,4.285714,0.000000,4.714286,0.0,0.0,0.0,0.0,0.000000,4.857143,4.571429
5531,273_yves-rocher,Repair Hair Mask,36357,50,289000.0,2022-10-27T07:33:19.380Z,Hair Mask,4.750000,5,5,1,0,4,58,Hair Care,Hair Treatment,Hair Mask,4.000000,0.000000,5.000000,0.0,0.0,0.0,0.0,0.000000,5.000000,5.000000


In [457]:
df.isna().sum()

brand_name                      0
product_name                    0
product_id                      0
beauty_point_earned             0
price_range                     0
active_date                     0
default_category                0
average_rating                  0
total_reviews                   0
total_recommended_count         0
total_repurchase_maybe_count    0
total_repurchase_no_count       0
total_repurchase_yes_count      0
total_in_wishlist               0
categories1                     0
categories2                     0
categories3                     0
star_value_for_money            0
star_long_wear                  0
star_texture                    0
star_quality                    0
star_eficiency                  0
star_durability                 0
star_scent                      0
star_pigmentation               0
star_packaging                  0
star_effectiveness              0
dtype: int64

In [458]:
import pandas as pd

# Assuming df is your original DataFrame

# Product Data
product_data_columns = ['product_id', 'brand_name', 'product_name', 'beauty_point_earned', 'price_range', 'active_date',
                        'default_category', 'total_in_wishlist', 'categories1', 'categories2', 'categories3']

product_data = df[product_data_columns]

# Rating Data
rating_data_columns = ['product_id', 'average_rating', 'total_reviews', 'total_recommended_count',
                       'total_repurchase_maybe_count', 'total_repurchase_no_count', 'total_repurchase_yes_count',
                       'star_value_for_money', 'star_long_wear', 'star_texture', 'star_quality', 'star_eficiency',
                       'star_durability', 'star_scent', 'star_pigmentation', 'star_packaging', 'star_effectiveness']

rating_data = df[rating_data_columns]

# save to csv
product_data.to_csv(r'/home/redha/Documents/projects/NLP/sociolla/artifacts/product_data.csv', index=False)
rating_data.to_csv(r'/home/redha/Documents/projects/NLP/sociolla/artifacts/rating_data.csv', index=False)
